In [ ]:
cd /content

/content


In [ ]:
!pwd

/content/drive/MyDrive/apache-spark-structured-streaiming-api--with-mongodb


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [902 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,

In [ ]:
!pip install findspark
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 67.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=aa55a5c41eb57795908745cf6467b268359a56022086d570a0f114b8363fb276
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.3.0-bin-hadoop3'

# Persit Apache Spark data to MongoDB

1. Create a function to write a row at a time to the database.
2. Use a streaming dataframe Writestream to output the data to a console
3. Use a streaming dataframe to output the data to MongoDB
4. Verify the MongoDB data

## Install MongoDB

In [21]:
# MongoDB download and installation
!wget https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-debian71-3.0.15.tgz  # Downloads MongoDB from official repository
!tar xfv mongodb-linux-x86_64-debian71-3.0.15.tgz     # Unpack compressed file
!rm mongodb-linux-x86_64-debian71-3.0.15.tgz          # Removes downloaded file

# Default location of database is "/data/db" folder  
!mkdir /data                                          # data folder creation 
!mkdir /data/db                                       # db folder creation inside data

--2022-08-06 15:52:03--  https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-debian71-3.0.15.tgz
Resolving fastdl.mongodb.org (fastdl.mongodb.org)... 13.224.2.75, 13.224.2.102, 13.224.2.108, ...
Connecting to fastdl.mongodb.org (fastdl.mongodb.org)|13.224.2.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70878938 (68M) [application/x-gzip]
Saving to: ‘mongodb-linux-x86_64-debian71-3.0.15.tgz’

mongodb-linux-x86_6 100%[===================>]  67.59M  24.6MB/s    in 2.7s    

2022-08-06 15:52:06 (24.6 MB/s) - ‘mongodb-linux-x86_64-debian71-3.0.15.tgz’ saved [70878938/70878938]

mongodb-linux-x86_64-debian71-3.0.15/README
mongodb-linux-x86_64-debian71-3.0.15/THIRD-PARTY-NOTICES
mongodb-linux-x86_64-debian71-3.0.15/GNU-AGPL-3.0
mongodb-linux-x86_64-debian71-3.0.15/bin/mongodump
mongodb-linux-x86_64-debian71-3.0.15/bin/mongorestore
mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport
mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport
mongodb-linux-x86_64-debi

In [22]:
spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/prcp.hpcp") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/prcp.hpcp") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

In [23]:
spark

In [24]:
def write_row(batch_df , batch_id):
    batch_df.write.format("mongo").mode("append").save()
    pass